In [1]:
import os, sys
from graphnet.data.sqlite.sqlite_utilities import create_table
import pandas as pd
import sqlite3
import pyarrow.parquet as pq
import sqlalchemy
from tqdm import tqdm
from typing import Any, Dict, List, Optional
import numpy as np
import pickle
import gc

graphnet: INFO     2023-02-28 00:39:28 - get_logger - Writing log to logs/graphnet_20230228-003928.log
graphnet: WARNING  2023-02-28 00:39:29 - warn_once - `icecube` not available. Some functionality may be missing.


In [2]:
input_data_folder = './data/train'
meta_data_path = './data/train_meta.parquet'
geometry_table = pd.read_csv('./data/sensor_geometry.csv')

In [3]:
def load_input(meta_batch: pd.DataFrame, input_data_folder: str) -> pd.DataFrame:
        batch_id = meta_batch['batch_id'].unique()
        assert len(batch_id) == 1, "contains multiple batch_ids. Did you set the batch_size correctly?"
        
        detector_readings = pd.read_parquet(path = f'{input_data_folder}/batch_{batch_id[0]}.parquet')
        sensor_positions = geometry_table.loc[detector_readings['sensor_id'], ['x', 'y', 'z']]
        sensor_positions.index = detector_readings.index

        for column in sensor_positions.columns:
            if column not in detector_readings.columns:
                detector_readings[column] = sensor_positions[column]

        detector_readings['auxiliary'] = detector_readings['auxiliary'].replace({True: 1, False: 0})
        return detector_readings.reset_index()


In [4]:
def add_to_table(database_path: str,
                      df: pd.DataFrame,
                      table_name:  str,
                      is_primary_key: bool,
                      engine: sqlalchemy.engine.base.Engine) -> None:
                      
    try:
        create_table(   columns=  df.columns,
                        database_path = database_path, 
                        table_name = table_name,
                        integer_primary_key= is_primary_key,
                        index_column = 'event_id')
    except sqlite3.OperationalError as e:
        if 'already exists' in str(e):
            pass
        else:
            raise e
   
    df.to_sql(table_name, con=engine, index=False, if_exists="append", chunksize = 200000)
    engine.dispose()
    return

In [5]:
def convert_to_sqlite(meta_data_path: str,
                      database_path: str,
                      input_data_folder: str,
                      batch_size: int = 200000,
                      batch_ids: list = [],
                      engine: sqlalchemy.engine.base.Engine = None
                      ) -> None:
    
    meta_data_iter = pq.ParquetFile(meta_data_path).iter_batches(batch_size = batch_size)
    batch_id = 1
    converted_batches = []
    for meta_data_batch in meta_data_iter:
        if batch_id in batch_ids:
            meta_data_batch  = meta_data_batch.to_pandas()
            add_to_table(database_path = database_path,
                        df = meta_data_batch,
                        table_name='meta_table',
                        is_primary_key= True,
                        engine = engine)
            pulses = load_input(meta_batch=meta_data_batch, input_data_folder= input_data_folder)
            del meta_data_batch 
            add_to_table(database_path = database_path,
                        df = pulses,
                        table_name='pulse_table',
                        is_primary_key= False,
                        engine = engine)
            del pulses 
            converted_batches.append(batch_id)
        batch_id +=1
        if len(batch_ids) == len(converted_batches):
            break
        gc.collect()
    del meta_data_iter 
    print(f'Conversion Complete! Database available at\n {database_path}')

In [6]:
# list_dict = {}
# list_train_ids = range(1,661)
# for batch_number in range(0,5):
#     list_dict[batch_number] = np.random.choice(list_train_ids, 110, replace=False)
#     list_train_ids = [x for x in list_train_ids if x not in list_dict[batch_number]]
#     print(f'Batch {batch_number} contains {len(list_dict[batch_number])} events')


# with open('5_big_batch_indx.pkl', 'wb') as f:
#     pickle.dump(list_dict, f)


In [7]:
list_dict = pd.read_pickle('5_big_batch_indx.pkl')

In [8]:
for batch_number in tqdm([0]):
    database_path = f'./data/extra_big_batch_{batch_number}.db'
    engine = sqlalchemy.create_engine("sqlite:///" + database_path)
    convert_to_sqlite(meta_data_path,
                    database_path=database_path,
                    input_data_folder=input_data_folder,
                    batch_size=200000,
                    batch_ids=list_dict[batch_number],
                    engine=engine)

100%|██████████| 1/1 [3:44:05<00:00, 13445.28s/it]

Conversion Complete! Database available at
 ./data/extra_big_batch_0.db
